In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

from astropy import units as u
from astropy import wcs
from astropy.io import fits
from astropy.table import Table
from astropy.nddata import NDData
from astropy.coordinates import SkyCoord
from astropy.modeling import models, fitting
from astropy.stats import SigmaClip, sigma_clipped_stats
from astropy.convolution import Gaussian2DKernel, Tophat2DKernel, convolve
from astropy.visualization import ZScaleInterval, MinMaxInterval, HistEqStretch
from astropy.visualization.mpl_normalize import ImageNormalize, simple_norm
from astropy.modeling.functional_models import Gaussian2D

from photutils import aperture, detection
from photutils.psf import (
    EPSFBuilder,
    IterativePSFPhotometry,
    extract_stars,
    make_psf_model,
)
from photutils.segmentation import detect_sources, make_2dgaussian_kernel
from photutils.background import Background2D, SExtractorBackground

# Configure matplotlib visualization parameters
plt.rcParams.update(
    {
        "figure.figsize": (8, 7),  # Default figure size
        "font.size": 10,  # Default font size
        "lines.linewidth": 1,  # Default line width
        "xtick.labelsize": 10,  # X-axis tick label size
        "ytick.labelsize": 10,  # Y-axis tick label size
        "axes.titlesize": 10,  # Axis title size
        "legend.fontsize": 10,  # Legend font size
    }
)

In [ ]:
def zscale_image(data, cmap="viridis"):
    """
    Affiche une image avec une échelle de couleur ajustée automatiquement en utilisant l'algorithme ZScaleInterval.

    Paramètres :
    data : array-like
        Les données de l'image à afficher.
    cmap : str, optionnel
        Le nom de la colormap à utiliser pour l'affichage. Par défaut, 'viridis'.
    """
    # Initialisation de l'algorithme ZScaleInterval pour déterminer les limites de l'échelle de couleur
    norm = ZScaleInterval()

    # Calcul des valeurs minimale et maximale pour l'échelle de couleur en utilisant les données fournies
    vmin, vmax = norm.get_limits(data)

    # Création de la figure et affichage de l'image avec la colormap spécifiée
    plt.figure()
    plt.imshow(
        data,
        vmin=vmin,  # Valeur minimale de l'échelle de couleur
        vmax=vmax,  # Valeur maximale de l'échelle de couleur
        origin="lower",  # Origine des coordonnées en bas à gauche
        cmap=cmap,
    )  # Colormap utilisée pour l'affichage
    plt.colorbar(aspect=8, shrink=0.75)  # Ajout d'une barre de couleur

    # Affichage des valeurs minimale et maximale de l'échelle de couleur dans la console
    print(f"vmin: {vmin}, vmax: {vmax}")

    # Affichage de l'image
    _ = plt.show()

In [ ]:
def histo_image(data, cmap="viridis"):
    """
    Affiche une image avec une normalisation et un étirement de l'histogramme.

    Paramètres :
    data : array-like
        Les données de l'image à afficher.
    cmap : str, optionnel
        Le nom de la colormap à utiliser pour l'affichage. Par défaut, 'viridis'.
    """
    # Initialisation de la normalisation de l'image avec MinMaxInterval et HistEqStretch
    norm = ImageNormalize(
        data,
        interval=MinMaxInterval(),  # Normalisation basée sur les valeurs min et max
        stretch=HistEqStretch(),
    )  # Étirement de l'histogramme des données

    # Création de la figure et affichage de l'image avec la normalisation et la colormap spécifiées
    plt.figure()
    plt.imshow(
        data,
        norm=norm,  # Application de la normalisation à l'image
        origin="lower",  # Origine des coordonnées en bas à gauche
        cmap=cmap,
    )  # Colormap utilisée pour l'affichage
    plt.colorbar(aspect=8, shrink=0.75)  # Ajout d'une barre de couleur

    # Affichage de l'image
    _ = plt.show()

In [ ]:
def airmass(head, latitude=48.29166, longitude=2.43805, elevation=95.0):
    """
    Calcule la masse d'air pour un objet céleste à des coordonnées données, un instant donné, et un lieu d'observation spécifique.

    Paramètres :
    head : dict
        Dictionnaire contenant les informations sur les coordonnées de l'objet céleste et la date d'observation.
    latitude : np.float, optionnel
        Latitude de l'observatoire en degrés. Par défaut, 48.29166.
    longitude : np.float, optionnel
        Longitude de l'observatoire en degrés. Par défaut, 2.43805.
    elevation : np.float, optionnel
        Élévation de l'observatoire en mètres. Par défaut, 95.0.

    Retourne :
    np.float
        La masse d'air pour l'objet céleste à l'instant et au lieu donnés.
    """
    try:
        ra = head.get("RA", head.get("OBJRA"))
        dec = head.get("DEC", head.get("OBJDEC"))
        obstime = head["DATE-OBS"]

        if ra is None or dec is None:
            raise KeyError("RA/DEC or OBJRA/OBJDEC not found in header.")

    except KeyError as e:
        print(f"Error: {e}")
        return 0

    # Création de l'objet SkyCoord avec les coordonnées fournies
    coord = SkyCoord(ra=ra, dec=dec, unit=u.deg, frame="icrs")

    # Création de l'objet Observer pour représenter l'observatoire
    observatory = Observer(
        latitude=latitude * u.deg,
        longitude=longitude * u.deg,
        elevation=elevation * u.m,
    )

    # Calcul de la masse d'air à partir des coordonnées altazimutales (altaz) de l'objet à l'heure d'observation
    am = observatory.altaz(obstime, coord).secz

    # Vérification et gestion des valeurs non physiques de la masse d'air
    if am < 1:
        print("Warning: Airmass is less than 1, check input data.")

    # Affichage et retour de la masse d'air calculée
    print(f"The airmass is: {am}")
    return am

In [ ]:
def image_quality(data, im_wcs, sigma=3.0, seeing=3.0):
    """
    Évalue la qualité d'une image astronomique en calculant des statistiques de base et en estimant la largeur à mi-hauteur (FWHM).

    Cette fonction calcule les statistiques de base (moyenne, médiane, écart-type) des données de l'image après application d'un sigma-clipping.
    Elle estime également la FWHM de l'image en pixels en utilisant l'échelle des pixels en arcsecondes fournie par le WCS de l'image et une valeur de seeing spécifiée.

    Paramètres :
    data (array-like) :
        Les données de l'image à analyser.
    im_wcs (astropy.wcs.WCS) :
        Le système de coordonnées du monde (WCS) associé à l'image.
    sigma (np.float, optionnel) :
        Le facteur de clipping pour le sigma-clipping des statistiques (par défaut 3.0).
    seeing (np.float, optionnel) :
        La valeur du seeing en arcsecondes pour estimer la FWHM (par défaut 3.0).

    Retourne :
    tuple :
        Un tuple contenant :
        - mean (np.float) : La valeur moyenne des données de l'image après sigma-clipping.
        - median (np.float) : La médiane des données de l'image après sigma-clipping.
        - std (np.float) : L'écart-type des données de l'image après sigma-clipping.
        - fwhm (np.float) : La largeur à mi-hauteur estimée en pixels.
        - pixel_scale (np.float) : L'échelle des pixels en arcsecondes.
    """
    # Affiche la valeur minimale des données de l'image en unités ADU (Analog-to-Digital Units)
    print(f"Min is : {np.min(data)} ADU")

    # Affiche la valeur maximale des données de l'image en unités ADU
    print(f"Max is : {np.max(data)} ADU")

    # Calcule les statistiques de base des données de l'image après sigma-clipping
    mean, median, std = sigma_clipped_stats(data, sigma=sigma)

    # Affiche l'écart-type des données de l'image après sigma-clipping
    print(f"StD : {std} ADU")

    # Affiche la valeur moyenne des données de l'image après sigma-clipping
    print(f"Mean : {mean} ADU")

    # Affiche la médiane des données de l'image après sigma-clipping
    print(f"Median : {median} ADU")

    # Calcule l'échelle des pixels en arcsecondes à partir du WCS de l'image
    pixel_scale = np.median(wcs.utils.proj_plane_pixel_scales(im_wcs)) * u.deg.to(
        u.arcsec
    )

    # Affiche l'échelle des pixels en arcsecondes
    print(f"Pixel Scale : {round(pixel_scale, 2)} arcsec")

    # Calcule la largeur à mi-hauteur (FWHM) en pixels en utilisant la valeur du seeing spécifiée
    fwhm = (seeing * u.arcsec) / pixel_scale

    # Affiche la FWHM estimée en pixels
    print(f"FWHM : {round(fwhm.value, 2)} pixels")

    # Retourne les statistiques calculées et la FWHM en pixels
    return mean, median, std, round(fwhm.value, 2), round(pixel_scale, 2)

In [ ]:
def load_fits(fpath, extension=0):
    """
    Charge un fichier FITS et retourne le header, les données, et un objet WCS si disponible.

    Paramètres :
    fpath (str): Chemin vers le fichier FITS.
    extension (int): Extension du fichier FITS à ouvrir (par défaut 0).

    Retourne :
    tuple: Contient le header, les données de l'extension spécifiée du fichier FITS,
           et un objet WCS si les informations WCS sont présentes, sinon None.
    """
    # Ouvre le fichier FITS et extrait les données et le header de l'extension spécifiée
    with fits.open(fpath, memmap=False) as hdu:
        header = hdu[extension].header
        data = hdu[extension].data

    # Vérifie si le header contient les informations WCS
    if any(
        key in header
        for key in [
            "CTYPE1",
            "CTYPE2",
            "CRVAL1",
            "CRVAL2",
            "CRPIX1",
            "CRPIX2",
            "CDELT1",
            "CDELT2",
            "CD1_1",
            "CD1_2",
            "CD2_1",
            "CD2_2",
        ]
    ):
        wcs_info = wcs.WCS(header)
    else:
        wcs_info = None
        print("Warning: No WCS information found in the FITS header.")

    return header, data, wcs_info

In [ ]:
def compute_background(
    img, sigma=3.0, box_size=(100, 100), filter_size=(5, 5), mask=None
):
    """
    Calcule le fond de ciel d'une image en utilisant la méthode `Background2D` de `photutils`.

    Paramètres :
    img (ndarray): Image pour laquelle le fond de ciel doit être estimé.
    sigma (np.float, optionnel): Seuil de coupure en sigma pour le sigma-clipping (par défaut 3.0).
    box_size (tuple, optionnel): Taille de la boîte pour le calcul du fond de ciel (par défaut (100, 100)).
    filter_size (tuple, optionnel): Taille du filtre pour lisser le fond de ciel (par défaut (5, 5)).
    mask (ndarray, optionnel): Masque binaire où les pixels masqués sont exclus du calcul du fond de ciel.

    Retourne :
    Background2D: Objet contenant le fond de ciel estimé et la carte du bruit.
    """
    # Initialise l'objet SigmaClip pour exclure les valeurs aberrantes
    sigma_clip = SigmaClip(sigma=sigma)

    # Initialise l'estimateur du fond de ciel basé sur SExtractor
    bkg_estimator = SExtractorBackground()

    # Calcule le fond de ciel et la carte du bruit avec Background2D
    bkg = Background2D(
        img,
        box_size=box_size,
        filter_size=filter_size,
        mask=mask,
        sigma_clip=sigma_clip,
        bkg_estimator=bkg_estimator,
        exclude_percentile=25,
    )

    return bkg

In [ ]:
def make_bord_mask(img, border=50):
    """
    Crée un masque binaire pour une image en excluant une bordure spécifiée.

    Paramètres :
    img (ndarray): Image pour laquelle le masque doit être créé.
    border (int, optionnel): Largeur de la bordure à exclure (par défaut 50 pixels).

    Retourne :
    ndarray: Masque binaire de la même taille que l'image, avec True pour les pixels inclus et False pour les pixels exclus.
    """
    # Initialise un masque de la même taille que l'image, avec tous les pixels masqués (False)
    mask = np.zeros(img.shape, dtype=bool)

    # Active les pixels à l'intérieur de la bordure spécifiée (True)
    mask[border : img.shape[0] - border, border : img.shape[1] - border] = True

    # Inverse le masque pour exclure la région intérieure et inclure la bordure
    return ~mask

In [ ]:
def fwhm_fit(img, sources, fwhm, pixel_scale, std_lo=1.5, std_hi=2.0):
    """
    Calcule la largeur à mi-hauteur (FWHM) d'une image en ajustant des modèles gaussiens et de Moffat aux sources détectées.

    Paramètres :
    img (ndarray): Image dans laquelle les sources ont été détectées.
    sources (Table): Table contenant les positions et les flux des sources détectées.
    fwhm (np.float): Estimation initiale de la FWHM pour l'ajustement.
    pixel_scale (np.float): Échelle en pixels pour la conversion des distances.

    Retourne :
    np.float: FWHM moyenne estimée en pixels, basée sur les modèles gaussiens et de Moffat.
    """
    # Filtrage des sources pour garder celles avec un flux dans une plage spécifique
    flux = sources["flux"]
    median_flux = np.median(flux)
    std_flux = np.std(flux)
    mask = (flux > median_flux + std_lo * std_flux) & (
        flux < median_flux + std_hi * std_flux
    )
    sources = sources[mask]

    # Suppression des sources contenant des valeurs NaN
    sources = sources[~np.isnan(sources["flux"])]

    print(f"Filtered Sources: {len(sources)}")

    # Initialisation du fitter
    fitter = fitting.SimplexLSQFitter()

    # Paramètres de la grille d'analyse
    analysis_radius = 3 * round(fwhm)

    # Variables pour stocker les résultats
    fwhm_values = []

    # Ajustement des modèles pour chaque source
    for x_cen, y_cen, peak in zip(
        sources["xcentroid"], sources["ycentroid"], sources["peak"]
    ):
        # Définir les limites de la sous-image
        x_start, x_end = int(x_cen) - analysis_radius, int(x_cen) + analysis_radius
        y_start, y_end = int(y_cen) - analysis_radius, int(y_cen) + analysis_radius

        # Extraction de la sous-image
        sub_img = img[y_start:y_end, x_start:x_end]

        # Génération des grilles de coordonnées pour l'ajustement
        x_grid, y_grid = np.meshgrid(
            np.arange(x_start, x_end), np.arange(y_start, y_end)
        )

        # Calcul des distances en pixels par rapport au centre de la source
        distances = np.sqrt((x_grid - x_cen) ** 2 + (y_grid - y_cen) ** 2)
        distances = distances.ravel()

        # Calcul des flux normalisés par le pic de chaque source
        flux_counts = (sub_img / peak).ravel()

        # Ajustement du modèle Gaussien
        gauss_model = models.Gaussian1D(amplitude=1.0, mean=0, stddev=fwhm)
        fitted_gauss = fitter(gauss_model, distances, flux_counts)
        fwhm_gauss = 2.355 * fitted_gauss.stddev * pixel_scale

        # Ajustement du modèle Moffat
        moffat_model = models.Moffat1D(amplitude=1.0, x_0=0, gamma=2.0, alpha=3.5)
        fitted_moffat = fitter(moffat_model, distances, flux_counts)
        fwhm_moffat = (
            fitted_moffat.gamma
            * 2
            * np.sqrt(2 ** (1.0 / fitted_moffat.alpha) - 1)
            * pixel_scale
        )

        # Ajout des résultats
        fwhm_values.extend([fwhm_gauss, fwhm_moffat])

    # Calcul de la FWHM moyenne
    mean_fwhm = np.array(fwhm_values).flatten()
    mean_fwhm = mean_fwhm[~np.isnan(mean_fwhm)]
    mean_fwhm = np.median(mean_fwhm[~np.isinf(mean_fwhm)])

    print(
        f"Median FWHM estimation based on Gaussian and Moffat models: {round(mean_fwhm)} pixels"
    )

    return round(mean_fwhm)

In [ ]:
def pretty_print(table, precision=8):
    """
    Formate et affiche une table avec une précision numérique spécifiée.

    Paramètres :
    table (astropy.table.Table): La table à afficher.
    precision (int, optionnel): Nombre de chiffres significatifs pour les valeurs numériques (par défaut 8).

    Retourne :
    astropy.table.Table: La table formatée.
    """
    # Définit le format d'affichage pour chaque colonne de la table
    for col in table.colnames:
        table[col].info.format = "%.{}g".format(
            precision
        )  # Formatage pour une précision uniforme

    # Affiche la table avec les types de données visibles
    table.pprint(show_dtype=True)

    return table

In [ ]:
header, img, img_wcs = load_fits("")
zscale_image(img)

In [ ]:
am = airmass(header)

In [ ]:
mean, median, std, fwhm, pixel_scale = image_quality(img, img_wcs, seeing=3.0)

In [ ]:
bkg = compute_background(img, sigma=3.0, box_size=(50, 50), filter_size=(5, 5))
zscale_image(bkg.background)

In [ ]:
mask = make_bord_mask(img, border=100)

In [ ]:
new_img = img.astype(np.float)
img_sub, bg = sep_subtract_background(new_img, bw=50, bh=50, fw=5, fh=5, fthresh=0.01)
zscale_image(bg.back())

In [ ]:
print(np.median(bkg.background - bg.back()), np.std(bkg.background - bg.back()))
zscale_image(img - bkg.background)

In [ ]:
obj, segmap = sep_extract_sources(img_sub, bg, mask=mask, sigma=1.5)

In [ ]:
plt.imshow(segmap, origin="lower")

In [ ]:
flux, fluxerr, flag = process_kron_aperture(img_sub, obj, mask=mask)

In [ ]:
sigma = 6.0
daofind = detection.DAOStarFinder(fwhm=2.0 * round(fwhm), threshold=sigma * std)
sources = daofind(img - bkg.background, mask=mask)

positions = np.transpose((sources["xcentroid"], sources["ycentroid"]))

# Tri des sources détectées par ordre décroissant de flux
sources.sort(keys="flux", reverse=True)

# Affichage du nombre de sources détectées
sources = pretty_print(sources, precision=8)

In [ ]:
# Bkg subtracted image
bkg_subtracted_img = img - bkg.background
i_fwhm = fwhm_fit(bkg_subtracted_img, sources, fwhm, pixel_scale)

In [ ]:
sigma = 3
daostarfind = detection.DAOStarFinder(fwhm=2.0 * i_fwhm, threshold=sigma * std)
final_sources = daostarfind(bkg_subtracted_img, mask=mask)

mask_valid = (
    ~np.isnan(final_sources["xcentroid"])
    & ~np.isnan(final_sources["ycentroid"])
    & ~np.isinf(final_sources["xcentroid"])
    & ~np.isinf(final_sources["ycentroid"])
)
final_sources = final_sources[mask_valid]

positions = np.transpose((final_sources["xcentroid"], final_sources["ycentroid"]))

# Tri des sources détectées par ordre décroissant de flux
final_sources.sort(keys="flux", reverse=True)

# Affichage du nombre de sources détectées
final_sources = pretty_print(final_sources, precision=8)

In [ ]:
radius = [1.0 * i_fwhm, 1.5 * i_fwhm, 2 * i_fwhm, 2.5 * i_fwhm, 3.0 * i_fwhm]
apertures = [aperture.CircularAperture(positions, r=r) for r in radius]

phot_table = aperture.aperture_photometry(img - bkg.background, apertures)
phot_table = pretty_print(phot_table, precision=8)

In [ ]:
def perform_psf_photometry(img, phot_table, std_x, std_y, fwhm, daostarfind):
    """
    Effectue une photométrie PSF sur une image.

    Parameters:
    - img (numpy.ndarray): Image avec le fond de ciel soustrait.
    - phot_table (astropy.table.Table): Table contenant les positions des sources.
    - std_x (np.float): Écart-type en x pour le modèle Gaussian2D.
    - std_y (np.float): Écart-type en y pour le modèle Gaussian2D.
    - fwhm (np.float): Full Width at Half Maximum pour ajuster la taille de l'ajustement.
    - daostarfind (callable): Fonction de détection des étoiles.

    Returns:
    - phot (astropy.table.Table): Résultats de la photométrie PSF.
    - resid (numpy.ndarray): Image des résidus après ajustement PSF.
    """

    # Définir le modèle PSF
    model = Gaussian2D(amplitude=1, x_stddev=std_x, y_stddev=std_y)
    # model = Moffat2D()
    psf_model = make_psf_model(model, use_dblquad=True)

    # Définir la forme de l'ajustement
    fit_shape = 2 * round(fwhm) + 1

    # Configurer la photométrie PSF
    psfphot = IterativePSFPhotometry(
        psf_model,
        fit_shape,
        finder=daostarfind,
        aperture_radius=fit_shape / 2,
        progress_bar=True,
        fitter=fitting.LMLSQFitter(),
    )

    # Spécifier les positions des sources
    psfphot.x = phot_table["xcenter"]
    psfphot.y = phot_table["ycenter"]

    # Créer un masque pour les valeurs non finies
    mask = ~np.isfinite(img)

    # Exécuter la photométrie PSF
    phot = psfphot(img, mask=mask)

    # Calculer l'image des modèles et des résidus
    # psf_img = psfphot.make_model_image(img, psf_shape=(fit_shape, fit_shape))
    psf_resid = psfphot.make_residual_image(img, psf_shape=(fit_shape, fit_shape))

    return phot, psf_resid


phot, psf_resid = perform_psf_photometry(
    bkg_subtracted_img, phot_table, std_x, std_y, i_fwhm, daostarfind
)
print(phot)

In [ ]:
def plot_psf_images(img1, img2):
    """
    Affiche les images de données, du modèle et des résidus en utilisant trois sous-plots dans une seule figure.

    Paramètres :
    img1 : array-like
        L'image avec le fond soustrait.
    img2 : array-like
        L'image des résidus.
    """
    # Création de la figure avec trois sous-plots
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5), sharex=True, sharey=True)

    # Plot de l'image de données
    cax1 = ax1.imshow(
        img1, norm=LogNorm(), cmap="viridis", origin="lower", interpolation="nearest"
    )
    ax1.set_title("Data")
    fig.colorbar(cax1, ax=ax1, orientation="vertical")

    # Plot de l'image du modèle
    cax2 = ax2.imshow(
        img1 - img2,
        norm=LogNorm(),
        cmap="viridis",
        origin="lower",
        interpolation="nearest",
    )
    ax2.set_title("Model")
    fig.colorbar(cax2, ax=ax2, orientation="vertical")

    # Plot de l'image des résidus
    cax3 = ax3.imshow(
        img2, norm=LogNorm(), cmap="viridis", origin="lower", interpolation="nearest"
    )
    ax3.set_title("Residual Image")
    fig.colorbar(cax3, ax=ax3, orientation="vertical")

    # Ajuste la mise en page pour éviter le chevauchement
    plt.tight_layout()

    # Affiche la figure
    _ = plt.show()


# Utilisation
plot_psf_images(bkg_subtracted_img, psf_resid)

In [ ]:
def perform_epsf_photometry(img, phot_table, fwhm, daostarfind, mask=None):
    """
    Effectue la photométrie PSF en utilisant un modèle EPSF.

    Parameters:
    - img (numpy.ndarray): Image avec le fond de ciel soustrait.
    - phot_table (astropy.table.Table): Table contenant les positions des sources.
    - fwhm (np.float): Full Width at Half Maximum pour ajuster la taille de l'ajustement.
    - daostarfind (callable): Fonction de détection des étoiles.
    - mask (numpy.ndarray): Masque pour les valeurs non finies dans l'image.

    Returns:
    - phot_epsf (astropy.table.Table): Résultats de la photométrie PSF avec le modèle EPSF.
    - epsf (photutils.epsf.EPSF): Modèle EPSF ajusté.
    """

    # Préparer les données
    nddata = NDData(data=img)
    stars_table = Table()
    stars_table["x"] = phot_table["xcenter"]
    stars_table["y"] = phot_table["ycenter"]

    # Définir la forme de l'ajustement
    fit_shape = 2 * round(fwhm) + 1

    # Extraire les étoiles
    stars = extract_stars(nddata, stars_table, size=fit_shape)

    # Afficher les images des étoiles extraites
    nrows = 5
    ncols = 5
    fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=(5, 5), squeeze=False)
    ax = ax.ravel()

    for i in range(min(len(stars), nrows * ncols)):
        norm = simple_norm(stars[i], "log", percent=99.0)
        ax[i].imshow(stars[i], norm=norm, origin="lower", cmap="viridis")
    _ = plt.show()

    # Ajuster le modèle EPSF
    epsf_builder = EPSFBuilder(oversampling=2, maxiters=3, progress_bar=True)
    epsf, fitted_stars = epsf_builder(stars)

    # Afficher le modèle EPSF ajusté
    norm = simple_norm(epsf.data, "log", percent=99.0)
    plt.figure()
    plt.imshow(
        epsf.data, norm=norm, origin="lower", cmap="viridis", interpolation="nearest"
    )
    plt.colorbar()
    _ = plt.show()

    # Effectuer la photométrie PSF
    psfphot = IterativePSFPhotometry(
        epsf,
        fit_shape,
        finder=daostarfind,
        aperture_radius=fit_shape / 2,
        progress_bar=True,
    )

    # Spécifier les positions des sources
    psfphot.x = phot_table["xcenter"]
    psfphot.y = phot_table["ycenter"]

    # Exécuter la photométrie PSF
    phot_epsf = psfphot(img, mask=mask)

    return phot_epsf, epsf


phot_epsf, epsf = perform_epsf_photometry(
    bkg_subtracted_img, phot_table, i_fwhm, daostarfind, mask
)
print(phot_epsf)

In [ ]:
def make_source_mask(data, nsigma, npixels, filter_fwhm=3.0, dilate_size=11):
    """
    Crée un masque de sources à partir de données en utilisant une convolution et une détection de sources.

    Parameters:
    - data (numpy.ndarray): Image ou tableau de données à analyser.
    - nsigma (np.float): Seuil en multiples d'écart-type pour la détection des sources.
    - npixels (int): Nombre minimum de pixels connectés pour qu'une détection soit considérée comme une source.
    - filter_fwhm (np.float, optional): Full Width at Half Maximum (FWHM) pour le noyau gaussien utilisé dans la convolution. La valeur par défaut est 3.0.
    - dilate_size (int, optional): Taille du masque de dilatation pour agrandir le masque des sources détectées. La valeur par défaut est 11.

    Returns:
    - numpy.ndarray: Masque binaire où les sources détectées sont marquées.
    """

    # Créer un noyau gaussien 2D pour la convolution
    kernel = make_2dgaussian_kernel(filter_fwhm, size=3)

    # Convoluer les données avec le noyau gaussien
    convolved_data = convolve(data, kernel)

    # Appliquer une coupure sigma pour filtrer les valeurs extrêmes
    sigma_clip = SigmaClip(sigma=3.0, maxiters=5)
    clipped_data = sigma_clip(data, masked=False)

    # Calculer la moyenne et l'écart-type des données coupées
    mean_ = np.nanmean(clipped_data)
    std_ = np.nanstd(clipped_data)

    # Définir le seuil pour la détection des sources basé sur nsigma
    threshold_2sigma = mean_ + nsigma * std_

    # Détecter les sources dans les données convoluées
    segm = detect_sources(convolved_data, threshold_2sigma, npixels)

    # Créer un masque de sources avec une dilatation
    return segm.make_source_mask(size=dilate_size)

In [ ]:
def dilate_mask(mask, tophat_size):
    """
    Dilate un masque binaire en utilisant un noyau top-hat 2D.

    Cette fonction agrandit les régions du masque binaire en utilisant la convolution avec un noyau top-hat.
    Les régions dilatées sont définies comme les zones où la valeur du masque convolué dépasse un seuil
    basé sur la taille du noyau.

    Parameters:
    - mask (numpy.ndarray): Masque binaire d'entrée à dilater. Les valeurs doivent être 0 ou 1.
    - tophat_size (np.float): Taille du noyau top-hat utilisé pour la dilatation. La taille est définie comme le diamètre du noyau.

    Returns:
    - numpy.ndarray: Masque dilaté, où les zones d'intérêt du masque d'origine sont étendues selon le noyau top-hat.
    """

    # Calculer la surface du noyau top-hat
    area = np.pi * tophat_size**2.0

    # Créer un noyau top-hat 2D avec la taille spécifiée
    kernel = Tophat2DKernel(tophat_size)

    # Convoluer le masque avec le noyau top-hat et appliquer le seuil
    # Les régions où la valeur convoluée dépasse 1/area sont considérées comme dilatées
    dilated_mask = convolve(mask, kernel) >= 1.0 / area

    return dilated_mask

In [ ]:
class SourceMask:
    """
    Classe pour créer et dilater un masque de sources à partir d'une image.

    Cette classe fournit des méthodes pour créer un masque de sources à différentes échelles
    et pour dilater ces masques en utilisant un noyau top-hat. Elle utilise les fonctions
    `make_source_mask` et `dilate_mask` pour ces opérations.

    Attributes:
    - img (numpy.ndarray): Image sur laquelle le masque de sources sera créé.
    - nsigma (np.float): Seuil en multiples d'écart-type pour la détection des sources.
    - npixels (int): Nombre minimum de pixels connectés pour qu'une détection soit considérée comme une source.
    """

    def __init__(self, img, nsigma=3.0, npixels=3):
        """
        Initialise l'objet SourceMask.

        Parameters:
        - img (numpy.ndarray): Image sur laquelle le masque sera basé.
        - nsigma (np.float, optional): Seuil en écarts-types pour la détection des sources. La valeur par défaut est 3.0.
        - npixels (int, optional): Nombre minimum de pixels connectés pour la détection. La valeur par défaut est 3.
        """
        self.img = img
        self.nsigma = nsigma
        self.npixels = npixels

    def single(self, filter_fwhm=3.0, tophat_size=5.0, mask=None):
        """
        Crée un masque de sources pour une seule échelle.

        Cette méthode crée un masque de sources à une échelle spécifiée par `filter_fwhm`
        et dilate ce masque en utilisant un noyau top-hat de taille `tophat_size`.

        Parameters:
        - filter_fwhm (np.float, optional): Full Width at Half Maximum pour le noyau gaussien utilisé dans la détection des sources. La valeur par défaut est 3.0.
        - tophat_size (np.float, optional): Taille du noyau top-hat utilisé pour dilater le masque. La valeur par défaut est 5.0.
        - mask (numpy.ndarray, optional): Masque binaire à soustraire de l'image avant la détection. La valeur par défaut est None.

        Returns:
        - numpy.ndarray: Masque dilaté pour une seule échelle.
        """
        if mask is None:
            image = self.img
        else:
            image = self.img * (1 - mask)

        # Créer le masque de sources à une échelle
        mask = make_source_mask(
            image,
            nsigma=self.nsigma,
            npixels=self.npixels,
            dilate_size=1,
            filter_fwhm=filter_fwhm,
        )

        # Dilater le masque créé
        return dilate_mask(mask, tophat_size)

    def multiple(self, filter_fwhm=[3.0], tophat_size=[3.0], mask=None):
        """
        Crée un masque de sources à plusieurs échelles.

        Cette méthode crée et dilate des masques de sources pour différentes échelles
        spécifiées par `filter_fwhm` et `tophat_size`. Les masques à chaque échelle
        sont combinés pour obtenir un masque final.

        Parameters:
        - filter_fwhm (list of np.float, optional): Liste des Full Width at Half Maximum pour les noyaux gaussiens utilisés dans la détection des sources. La valeur par défaut est [3.0].
        - tophat_size (list of np.float, optional): Liste des tailles des noyaux top-hat utilisés pour dilater les masques. La valeur par défaut est [3.0].
        - mask (numpy.ndarray, optional): Masque binaire à soustraire de l'image avant la détection. La valeur par défaut est None.

        Returns:
        - numpy.ndarray: Masque combiné dilaté pour toutes les échelles spécifiées.
        """
        if mask is None:
            self.mask = np.zeros(self.img.shape, dtype=bool)

        # Combiner les masques à différentes échelles
        for fwhm, tophat in zip(filter_fwhm, tophat_size):
            smask = self.single(filter_fwhm=fwhm, tophat_size=tophat, mask=mask)
            self.mask = self.mask | smask  # Fusionner les masques à chaque itération

        return self.mask

In [ ]:
def find_worst_residual_near_center(resid):
    """
    Trouve la position du pixel avec le pire résidu près du centre, en évitant les bords.

    Cette fonction recherche le pixel avec le plus grand résidu (valeur maximale) dans une région circulaire
    centrée dans l'image, en évitant les bords de l'image. La région considérée est définie par un rayon
    proportionnel à la taille de l'image.

    Parameters:
    - resid (numpy.ndarray): Image des résidus où chaque pixel représente un résidu calculé.

    Returns:
    - tuple: Coordonnées (ligne, colonne) du pixel avec le pire résidu dans la région définie.
    """

    # Calculer les coordonnées du centre de l'image
    yc, xc = resid.shape[0] / 2.0, resid.shape[1] / 2.0

    # Définir le rayon de la région circulaire autour du centre pour éviter les bords
    radius = resid.shape[0] / 3.0

    # Créer des indices pour chaque pixel dans l'image
    y, x = np.mgrid[0 : resid.shape[0], 0 : resid.shape[1]]

    # Créer un masque circulaire pour la région autour du centre
    mask = np.sqrt((y - yc) ** 2 + (x - xc) ** 2) < radius

    # Appliquer le masque à l'image des résidus
    rmasked = resid * mask

    # Trouver l'indice du pixel avec la valeur maximale dans la région masquée
    return np.unravel_index(np.argmax(rmasked), resid.shape)

In [ ]:
def plot_mask(scene, bkgd, mask, zmin, zmax, worst=None, smooth=0):
    """
    Crée un graphique en trois panneaux pour visualiser le masque et son impact sur l'image.

    Les trois panneaux montrent :
    1. Le masque appliqué à l'ensemble de l'image.
    2. L'image `scene` après soustraction de l'image `bkgd` et application du masque.
    3. Une région zoomée de l'image avec le masque superposé en contours.

    Parameters:
    - scene (numpy.ndarray): Image principale sur laquelle le masque est appliqué.
    - bkgd (numpy.ndarray): Image de fond à soustraire de l'image principale.
    - mask (numpy.ndarray): Masque binaire à appliquer à l'image.
    - zmin (np.float): Valeur minimale pour l'échelle de couleurs de l'image.
    - zmax (np.float): Valeur maximale pour l'échelle de couleurs de l'image.
    - worst (tuple of int, optional): Coordonnées (x, y) du pixel avec le pire résidu à mettre en évidence. Si None, le pire résidu est déterminé automatiquement.
    - smooth (np.float, optional): Taille du noyau gaussien pour lisser l'image après soustraction. Si 0, le lissage est désactivé. La valeur par défaut est 0.

    Returns:
    - tuple: Coordonnées (x, y) du pixel avec le pire résidu, ou celles fournies par `worst` si spécifié.
    """

    # Déterminer les coordonnées du pire résidu
    if worst is None:
        y, x = find_worst_residual_near_center(bkgd - np.std(bkgd))
    else:
        x, y = worst

    # Créer une figure avec trois sous-graphes
    plt.figure(figsize=(20, 10))

    # Premier panneau : afficher le masque
    plt.subplot(131)
    plt.imshow(mask, vmin=0, vmax=1, cmap=plt.cm.gray, origin="lower")
    plt.title("Mask")

    # Deuxième panneau : afficher la scène après soustraction et application du masque
    plt.subplot(132)
    if smooth == 0:
        plt.imshow((scene - bkgd) * (1 - mask), vmin=zmin, vmax=zmax, origin="lower")
    else:
        # Appliquer un lissage gaussien si spécifié
        smoothed = convolve((scene - bkgd) * (1 - mask), Gaussian2DKernel(smooth))
        plt.imshow(
            smoothed * (1 - mask),
            vmin=zmin / smooth,
            vmax=zmax / smooth,
            origin="lower",
        )
    plt.title("Scene with Mask Applied")

    # Troisième panneau : afficher la scène avec le masque en contours
    plt.subplot(133)
    plt.imshow(scene - bkgd, vmin=zmin, vmax=zmax)
    plt.contour(mask, colors="red", alpha=0.2)
    plt.title("Zoomed-In with Mask Contours")

    # Retourner les coordonnées du pire résidu
    return x, y

In [ ]:
mask_3sigma = make_source_mask(
    bkg_subtracted_img, nsigma=3, npixels=5, dilate_size=5, filter_fwhm=3
)

zmin, zmax = np.percentile(bkg.background, (0.1, 99.9))
sm = SourceMask(bkg_subtracted_img, nsigma=1.5)
mask = sm.multiple(filter_fwhm=[1, 3, 5], tophat_size=[4, 2, 1])

plot_mask(img, bkg.background, mask, zmin, zmax)

bkg2 = compute_background(
    img, sigma=3.0, box_size=(35, 50), filter_size=(5, 7), mask=mask_3sigma
)